In [12]:
import ee
import os
import fiona
import geemap
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET

In [13]:
# Initialise
ee.Authenticate()
ee.Initialize(project="jameswilliamchamberlain")

In [14]:
# basemap 
basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

# centeral point of Samarra 
df_sites = pd.DataFrame({
    "longitude": [43.823543],
    "latitude": [34.340989],
    "name": ["Samarra Archaeological City"],
    "category": ["Cultural"],
    "date inscribed": ["2007"],
    "region": ["Arab States"],
    "url": ["https://whc.unesco.org/en/list/276"],
    "iso": [["IQ"]]
})

# Chunks of Samarra Archaeological City 
with fiona.open("chunks_new.shp") as src:
    chunks = gpd.GeoDataFrame.from_features(src, crs=src.crs)

In [15]:
df_sites.head()

,longitude,latitude,name,category,date inscribed,region,url,iso
0,43.823543,34.340989,Samarra Archaeological City,Cultural,2007,Arab States,https://whc.unesco.org/en/list/276,[IQ]


In [16]:
m = geemap.Map()

if df_sites.empty:
    print("No sites found for the specified URL.")
else:
    m.add_points_from_xy(df_sites, x="longitude", y="latitude", layer_name="Sites")
    center_points = df_sites[['longitude', 'latitude']].mean().values
    m.setCenter(center_points[0], center_points[1], 10)

m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

# add chunks from aoi 
m.add_gdf(chunks, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})

m

Map(center=[34.340989, 43.823543], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [17]:
# first polygon 
first_polygon = chunks.geometry.iloc[0]

In [18]:
# create centroids at each four corners of the polygon (NOT BOUNDS)
# EXTRACT ALL POINTS
points = first_polygon.exterior.coords[:]

# drop duplicate points
points = set(points)

print("Points in the first polygon:"
      f"\n{points}")

# top most two
top_most = sorted(points, key=lambda x: x[1], reverse=True)[:2]

# left most two
left_most_two = sorted(points, key=lambda x: x[0])[:2]

# create ee.Geometry.Point objects for the centroids
centroids_left = [ee.Geometry.Point(lon, lat) for lon, lat in left_most_two]
centroids_top = [ee.Geometry.Point(lon, lat) for lon, lat in top_most]

# add to map
m.add_layer(ee.FeatureCollection(centroids_left), {}, "Centroids Left")
m.add_layer(ee.FeatureCollection(centroids_top), {}, "Centroids Top")

# get distance between the two centroids
distance_left = centroids_left[0].distance(centroids_top[0]).getInfo()
print(f"Distance between the leftmost two centroids: {distance_left} meters")

distance_top = centroids_top[0].distance(centroids_left[0]).getInfo()
print(f"Distance between the topmost two centroids: {distance_top} meters")


Points in the first polygon:
{(43.91574412629632, 34.111412926292765), (43.91285150660586, 34.336837086609854), (44.1846103461893, 34.3389484695521), (44.18678011312121, 34.11350662400319)}
Distance between the leftmost two centroids: 25007.629068888167 meters
Distance between the topmost two centroids: 25007.629068888167 meters


In [ ]:
def create_subregions(chunks, sift_percentage=0.5):
    """
        Shifts all polygons to 8 positions based on half the length in longitude and latitude, to create subregions - one for each direction from the center.

        Assumes aoi contains all similar polygons, and are similar to a square.

        Parameters:
            aoi (GeoDataFrame): The area of interest containing geometries.
            sift_percentage (float): Percentage of the length to shift the center point. Default is 0.5 (50%) for half the length.
    """

    if chunks.empty:
        return {}
    
    first_polygon = chunks.geometry.iloc[0]

    # take top two points of the polygon and get the length between them
    top_points = first_polygon.exterior.coords[:2]
    length_lon = abs(top_points[0][0] - top_points[1][0])

    # calculate the shift amount
    shift_amount = length_lon * sift_percentage

    shift_directions = {
        "left": (-shift_amount, 0),
        "right": (shift_amount, 0),
        "up": (0, shift_amount),
        "down": (0, -shift_amount),
        "top_left": (-shift_amount, shift_amount),
        "top_right": (shift_amount, shift_amount),
        "bottom_left": (-shift_amount, -shift_amount),
        "bottom_right": (shift_amount, -shift_amount),
    }

    subregions = []

    # Create subregions by shifting the geometries in all directions
    for _, (dx, dy) in shift_directions.items():
        gdf_shifted = chunks.copy()
        gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)
        subregions.append(gdf_shifted)

    # Combine all into one GeoDataFrame
    subregions = pd.concat(subregions, ignore_index=True)
    return gpd.GeoDataFrame(subregions, crs=chunks.crs)

def clip(chunks, aoi):
    """
        Clips the chunks or subregions to the area of interest (aoi).

        Parameters:
            chunks (GeoDataFrame):      The chunks or subregions to be clipped.
            aoi (GeoDataFrame):         The area of interest (aoi) to clip the chunks against.
    """

    # clip to aoi 
    if chunks.empty or aoi.empty:
        return gpd.GeoDataFrame(columns=chunks.columns.tolist() + ['direction'], crs=chunks.crs)
    clipped = gpd.clip(chunks, aoi)
    clipped = clipped[clipped.geometry.notnull()]  # Remove any null geometries

    return clipped.reset_index(drop=True)

subregions = create_subregions(chunks)
subregions = clip(subregions, aoi=chunks.dissolve())

# plot as one 
m.add_gdf(subregions, layer_name="Subregions", style={"color": "blue", "fillColor": "blue", "fillOpacity": 0.1})

In [ ]:
# Clustering 